In [20]:
import os
import json
import sqlite3
import pandas as pd
import shutil
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
data_igtao = datetime.now().strftime('%Y-%m-%d')

caminho_backup = '/workspaces/projeto_ing/INGESTION/app/src/backp/'
ctrl_ing = "/workspaces/projeto_ing/INGESTION/app/src/archiving_ctrl/arquivo_controle.txt"
landing = '/workspaces/projeto_ing/INGESTION/app/src/landing/'
layouts = '/workspaces/projeto_ing/INGESTION/app/src/tmp/layouts/'

In [21]:
read_landing = os.listdir(landing)
print("Conteúdos do diretório 'landing':", read_landing)
read_layouts = os.listdir(layouts)
print("Conteúdos do diretório 'layouts':", read_layouts)
print('===============================================')

Conteúdos do diretório 'landing': ['resultado_teste2_20242306_215405.txt']
Conteúdos do diretório 'layouts': ['teste']


In [30]:
for item in read_layouts:
    accept = os.path.join(layouts, item)
    if os.path.isdir(accept):
        print('========================')
        print(f'item encontrado no diretório: {item}')
        subconteudos = os.listdir(accept)
        print('========================')
        print(f' conteudo: {subconteudos}')
        
        caminho_estrutura_json = os.path.join(accept, 'estrutura.json')
        if os.path.exists(caminho_estrutura_json):
            print(f'Encontrado estrutura.json em {item}')
            
            with open(caminho_estrutura_json, 'r') as f:
                    config_estrutura = json.load(f)
                    print("Configuração estrutura:", config_estrutura)
                    
                    for estr in config_estrutura:
                        nm_arq = estr['nome_arquivo']
                        key = estr['chave']['palavra_chave']
                        banco = estr['chave']['banco']
                        tabela = estr['chave']['tabela']
                        print('=======================')
                        print(estr)
                        print(nm_arq)
                        print(key)
                        print(banco)
                        print(tabela)
                        print('=======================')
                        
                        for root, dirs, lista in os.walk(landing):
                            print(f'Procurando em {root}...')
                            for nome_arquivo in lista:
                                print(f'Verificando arquivo: {nome_arquivo}')
                                if key in nome_arquivo:
                                    caminho_completo = os.path.join(root, nome_arquivo)
                                    print(f'Arquivo contendo a chave encontrado: {caminho_completo}')
                                    sav = os.path.basename(caminho_completo)
                                    print(f'Conteúdo a gravar: {sav}')
                                    
                                    parte_util = sav.split('_')[1]
                                    print(f'Separando nome do arquivo: {parte_util}')
                        

item encontrado no diretório: teste
 conteudo: ['estrutura.json', 'tabela_teste2.json', 'tabela_teste.json']
Encontrado estrutura.json em teste
Configuração estrutura: [{'nome_arquivo': 'resultado_teste_', 'descricao': 'tabela teste', 'chave': {'palavra_chave': 'teste', 'banco': 'banco_teste.db', 'tabela': 'tabela_teste'}}, {'nome_arquivo': 'resultado_teste2_', 'descricao': 'tabela teste 2', 'chave': {'palavra_chave': 'teste2', 'banco': 'banco_teste.db', 'tabela': 'tabela_teste_2'}}]
{'nome_arquivo': 'resultado_teste_', 'descricao': 'tabela teste', 'chave': {'palavra_chave': 'teste', 'banco': 'banco_teste.db', 'tabela': 'tabela_teste'}}
resultado_teste_
teste
banco_teste.db
tabela_teste
Procurando em /workspaces/projeto_ing/INGESTION/app/src/landing/...
Verificando arquivo: resultado_teste2_20242306_215405.txt
Arquivo contendo a chave encontrado: /workspaces/projeto_ing/INGESTION/app/src/landing/resultado_teste2_20242306_215405.txt
Conteúdo a gravar: resultado_teste2_20242306_215405.tx